<a href="https://colab.research.google.com/github/BenzterBit/MLP/blob/master/AnimeRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"benzybit","key":"2d9bccaa7d4d615e650a0b725f35bb7e"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database -p/content
!unzip \*.zip

100% 25.2M/25.2M [00:00<00:00, 63.4MB/s]

Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


In [60]:

import pandas as pd
import numpy as np
anime_df = pd.read_csv('anime.csv')
for index, row in anime_df.iterrows():
  if row['rating']<6.50:
    anime_df =anime_df.drop([index], axis = 0)
    
rating_df = pd.read_csv('rating.csv')
#rating_df.head(20)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [61]:
anime_df['genre'] = anime_df.genre.str.split(',')
anime_df = anime_df.drop(['type','rating','episodes','members'],axis=1)
anime_df = anime_df.dropna()
anime_df.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]"
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,..."
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur..."
3,9253,Steins;Gate,"[Sci-Fi, Thriller]"
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur..."


In [62]:
anime_genre_df = anime_df.copy()

for index, row in anime_df.iterrows():
  for gen in row['genre']:
    anime_genre_df.at[index,gen]=1
anime_genre_df = anime_genre_df.fillna(0)
anime_genre_df.head()

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,Josei,Military,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
rating_df.head()
rating_df.shape

ratingless_df = rating_df.tail(250000)
ratingless_df.shape

(250000, 3)

In [0]:
for index, row in ratingless_df.iterrows():
  if row['rating']==-1:
    ratingless_df =ratingless_df.drop([index], axis = 0)
  

In [66]:
ratingless_df.shape

(202656, 3)

In [67]:
ratingless_df = ratingless_df['rating'].astype(dtype = 'float32')
ratingless_df.head(30)

7563737     1.0
7563738     7.0
7563739     6.0
7563740     8.0
7563741     2.0
7563742     5.0
7563743     7.0
7563744    10.0
7563745     7.0
7563746     5.0
7563747     7.0
7563748    10.0
7563749     6.0
7563750    10.0
7563751     7.0
7563752    10.0
7563753     9.0
7563754    10.0
7563755     6.0
7563756     8.0
7563757     6.0
7563758     7.0
7563759     7.0
7563760     6.0
7563761     6.0
7563762     6.0
7563763     5.0
7563764     8.0
7563765    10.0
7563766     9.0
Name: rating, dtype: float32

In [68]:
userInput = [
            {'name':'Kimi no Na wa.', 'rating':9},
            {'name':'Fullmetal Alchemist: Brotherhood', 'rating':9.0},
            {'name':'Steins;Gate', 'rating':9.5},
            {'name':"Death Note", 'rating':10},
            {'name':'Mob Psycho 100', 'rating':8.5},
            {'name':'Sword Art Online', 'rating':8.0},
            {'name':'Naruto', 'rating':9.0},
            {'name':'Tokyo Ghoul', 'rating':8.0},
            {'name':'Koe no Katachi', 'rating':9.0},
            {'name':'One Punch Man', 'rating':10.0},
            {'name':'Shokugeki no Souma', 'rating':9.0},
            {'name':'Bakuman. 2nd Season', 'rating':8.5},
            {'name':'Shingeki no Kyojin: Kuinaki Sentaku', 'rating':9.5},
            {'name':'Diamond no Ace', 'rating':9.0},
            {'name':'Magi: The Labyrinth of Magic', 'rating':8.5},
            {'name':'Noragami', 'rating':8.5},
         ] 
input_movies = pd.DataFrame(userInput)


input_movies.head()

,name,rating
0,Kimi no Na wa.,9.0
1,Fullmetal Alchemist: Brotherhood,9.0
2,Steins;Gate,9.5
3,Death Note,10.0
4,Mob Psycho 100,8.5


In [69]:
input_id = anime_df[anime_df['name'].isin(input_movies['name'])]
input_id = input_id.drop('genre',1)
input_movies = pd.merge(input_id,input_movies)
input_movies

,anime_id,name,rating
0,32281,Kimi no Na wa.,9.0
1,5114,Fullmetal Alchemist: Brotherhood,9.0
2,9253,Steins;Gate,9.5
3,28851,Koe no Katachi,9.0
4,30276,One Punch Man,10.0
5,1535,Death Note,10.0
6,28171,Shokugeki no Souma,9.0
7,32182,Mob Psycho 100,8.5
8,10030,Bakuman. 2nd Season,8.5
9,25781,Shingeki no Kyojin: Kuinaki Sentaku,9.5


In [70]:
user_anime = anime_genre_df[anime_genre_df['anime_id'].isin(input_movies['anime_id'])]
user_anime

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,Josei,Military,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,28851,Koe no Katachi,"[Drama, School, Shounen]",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,30276,One Punch Man,"[Action, Comedy, Parody, Sci-Fi, Seinen, ...",0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1535,Death Note,"[Mystery, Police, Psychological, Supernatur...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,28171,Shokugeki no Souma,"[Ecchi, School, Shounen]",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,32182,Mob Psycho 100,"[Action, Comedy, Slice of Life, Supernatural]",0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,10030,Bakuman. 2nd Season,"[Comedy, Drama, Romance, Shounen]",0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,25781,Shingeki no Kyojin: Kuin

In [71]:
user_anime = user_anime.reset_index(drop=True)
user_genre_table =user_anime.drop(['anime_id','name','genre'],1)
user_genre_table

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,Josei,Military,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
user_profile = user_genre_table.T.dot(input_movies['rating'])
user_profile

Drama             18.0
 Romance          25.5
 School           36.0
 Supernatural     63.5
Action            79.0
 Adventure        34.0
 Drama            25.5
 Fantasy          35.0
 Magic            17.5
 Military          9.0
 Shounen          70.5
 Comedy           27.5
 Historical        0.0
 Parody           10.0
 Samurai           0.0
 Sci-Fi           10.0
Sci-Fi             9.5
 Thriller         19.5
Comedy            17.5
 Sports            9.0
 Super Power      19.0
 Space             0.0
 Slice of Life     8.5
 Mecha             0.0
Adventure          0.0
 Music             0.0
 Mystery           8.0
 Seinen           18.0
Fantasy            0.0
 Martial Arts      9.0
                  ... 
Dementia           0.0
 Harem             0.0
Music              0.0
Game               0.0
 Cars              0.0
 Dementia          0.0
Cars               0.0
Mecha              0.0
Horror             0.0
School             0.0
 Kids              0.0
Historical         0.0
Kids       

In [73]:
genre_table = anime_genre_df.set_index(anime_genre_df['anime_id'])
genre_table = genre_table.drop(['anime_id','name','genre'],1)
genre_table.head()

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,Josei,Military,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Hentai,Yaoi,Hentai,Yuri,Yaoi
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
recom_table_df = ((genre_table*user_profile).sum(axis=1)/user_profile.sum())
recom_table_df.head(20)

anime_id
32281    0.219493
5114     0.415196
28977    0.302379
9253     0.044513
9969     0.302379
32935    0.243285
11061    0.310821
820      0.056792
15335    0.302379
15417    0.302379
4181     0.231005
28851    0.191097
918      0.302379
2904     0.218726
28891    0.243285
199      0.136608
23273    0.230238
24701    0.204144
12355    0.013047
1575     0.234843
dtype: float64

In [75]:
recom_table_sort_df = recom_table_df.sort_values(ascending=False)
recom_table_sort_df.head(10)

anime_id
4938     0.550269
25157    0.544129
6811     0.541059
249      0.541059
969      0.523408
177      0.498849
1536     0.478127
3588     0.475058
808      0.472755
31821    0.471988
dtype: float64

In [76]:
anime_df.loc[anime_df['anime_id'].isin(recom_table_sort_df.head(10).keys())]


,anime_id,name,genre
173,6811,InuYasha: Kanketsu-hen,"[Action, Adventure, Comedy, Demons, Fantas..."
286,4938,Tsubasa: Shunraiki,"[Action, Adventure, Drama, Fantasy, Magic,..."
440,3588,Soul Eater,"[Action, Adventure, Comedy, Fantasy, Shoun..."
690,249,InuYasha,"[Action, Adventure, Comedy, Demons, Fantas..."
1051,31821,Arslan Senki (TV): Fuujin Ranbu,"[Action, Adventure, Drama, Fantasy, Histor..."
1071,969,Tsubasa Chronicle 2nd Season,"[Action, Adventure, Drama, Fantasy, Myster..."
1129,177,Tsubasa Chronicle,"[Action, Adventure, Fantasy, Magic, Romanc..."
1522,1536,Busou Renkin,"[Action, Comedy, Fantasy, School, Shounen,..."
1845,25157,Trinity Seven,"[Action, Comedy, Ecchi, Fantasy, Harem, M..."
4432,808,Bakuretsu Hunters OVA,"[Action, Adventure, Comedy, Ecchi, Fantasy..."
